<a href="https://colab.research.google.com/github/jyeongvv/DL_study/blob/main/jyeongvv/0404_%EC%98%A4%ED%86%A0%EC%9D%B8%EC%BD%94%EB%8D%94_GAN_cGAN_%EB%B3%B5%EC%8A%B5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 오토인코더 (autoencoder)

* 실생활에 있는 개념으로 비유
* 다른 언어나 인접 영역의 다른 개념과 연결



---

* 비지도학습
* 인코더, 디코더, 잠재 변수
* 차원 축소, 특징 추출, 잡음 제거
* 재구성 오차, MSE

## 모듈
* 인코더, 디코더 두 부분
* nn.Sequential
  * 여러 모듈을 하나의 모듈로 묶음
  * 각 레이어를 데이터가 순차적으로 지나갈 때 사용하면 간결해짐
  * 계층과 활성화 함수를 정의해주면 **순서대로** 값을 처리해줌
* **인코더**는 간단한 신경망
   * 한줄로 줄이는 평탄화를 진행
   * 3차원에서 시각화 할 수 있도록 특징을 3개만 남김
   * 출력값 = 잠재변수
* **디코더**는 이미지 복원
  * 인코더를 거꾸로 만든것과 유사
  * 차원이 점점 증가
  * 차이점 : 마지막 출력값을 0~1 사이로 만들어주는 `Sigmoid` 함수가 추가됨


* 인코더(self.encoder)는 잠재변수인 encoded를 반환
* 디코더(self.decoder)는 복원 이미지인 decoded를 반환
---
* 최적화 함수 `Adam()` : SGD의 변형 함수 / 학습 중인 기울기를 참고하여 학습 속도를 자동으로 변화시킴
* 오차 함수 `nn.MSELoss` : 두 개의 같은 크기의 행렬을 받아, 각 자리의 차이에 제곱해서 평균을 구해주는 객체를 생성


* `view_data = view_data.type(torch.FloatTensor)/255.` .을 넣어주면 명시적으로 들어감 # 픽셀의 색상값이 `0~255` -> 우리가 만드는 모델은 0~1를 인식
* 오차값 : 디코더에서 나온 복원 이미지 decoded와 원본 이미지 y -> 차이의 제곱
* `optimizer.step()` -> 최적화 진행 

## 복원

* 시각화 -> 복원 이미지 (디코더에서 나온 이미지)
* `view_data.data.to("cpu")` : DEVICE로 보냈던 view_data를 cpu로 가져오게씀
* `numpy()` : tensor 데이터를 numpy array (ndarray) // matpoltlib으로 시각화하려면 tensor X, ndarray
* [i]는 각각 열에 한 개씩의 이미지를 연결

## 잠재 변수

* `test_x = view_data.to(DEVICE)` -> 모델이 있는 DEVICE로 보내기
* **`encoded_data` : 잠재변수, `decoded_data` : 복원된 이미지** 

## 망가진 이미지 복원

* 오토인코더는 '압축'을 진행
  * 압축 = 데이터으 ㅣ특성에 우선순위를 매기고, 낮은 순위의 데이터를 버림
* 잡음제거 `denoising`
  * 중요한 특징을 추출하는 오토인코더의 특성을 이용 -> 비교적 덜 중요한 데이터를 제거
  * 학습할 때 입력에 잡음을 더하는 방식으로 복원 능력 강화

---
* 하이퍼파라미터 진행
* 데이터셋 불러오기
* 모듈 돌려주기
* 노이즈 더하기
```
def add_noise(img): # img <- 파이토치의 텐서
    noise = torch.randn(img.size()) * 0.2 # 랜덤으로 생성된 의미 없는 값
    # -0.4 ~ +0.4
    noisy_img = img + noise
    return noisy_img # 색상값에 노이즈만큼 (정규분포확률로 생성된) 왜곡된 이미지
```
* 훈련

---

# GAN

## 생성자와 판별자

### 생성자

> 실제 데이터와 비슷한 가짜 데이터를 만들어 내는 **신경망**

* 정규분포로 뽑은 64차원의 무작위 텐서를 입력받아 Linear과 활성화 함수(ReLU, Tanh) 연산을 실행해줌
* Tanh은 결과값을 -1과 1사이로 압축
* 결과값 = 이미지 = 784차원
* 무작위 텐서 입력 이유 = 생성자가 실제 데이터의 '분포'를 배움
  * 생성자는 정규분포 같은 단순한 분포에서부터 실제 데이터의 복잡한 분포를 배움

### 판별자
* 이미지의 크기인 784차원의 텐서를 입력 받음
* 입력된 데이터에 행렬곱과 활성화 함수 실행
* 판별자는 입력된 텐서가 생성자가 만든 가짜 이미지인지, 실제 데이터인지 구분하는 **분류** 모델
* 판별자에서는 ReLU  대신 **Leaky ReLU** 사용
* **Leaky ReLU** 는 양의 기울기만 전달하는 ReLU와 다르게 음의 기울기도 전달함
  * 판별자에서 계산된 기울기가 0이 아니라 음수로 전환되며 생성자에게 더 강하게 전달됨
* 중요함니다

## 학습 구현

> 진짜, 가짜 레이블 생성
  * `real_labels = torch.ones(BATCH_SIZE, 1).to(DEVICE)` -> 1로 채워진 텐서 ( 행: 배치사이즈, 열: 1)
  * `fake_labels = torch.zeros(BATCH_SIZE, 1).to(DEVICE)`-> 0로 채워진 텐서 ( 행: 배치사이즈, 열: 1)

> 무작위 텐서로 가짜 이미지 생성
  * `z = torch.randn(BATCH_SIZE, 64).to(DEVICE)` -> 정규분포를 따르는 64개의 특성을 가진 가짜 이미지 텐서

> 진짜와 가짜 이미지를 가지고 낸 오차
  * d_loss = d_loss_real + d_loss_fake

> `d_optimizer.step()` -> 판별자의 패러미터 개선

* ~판별자가 속았는지를 봐서 판별자의 성능 개선
---
> 생성자가 판별자를 속였는지에 대해 (생성자 성능) 오차를 계산
  * fake_images = G(z)
  * outputs = D(fake_images)
  * 생성자가 속였는지
    * g_loss = criterion(outputs, real_labels)

> `g_optimizer.step()` -> 생성자의 패러미터 개선

---

# cGAN

* GAN과 크게 다르지 않음

> 다른점 
  * tensor 100개 input -> 110개 -> 100개 + 10개의 라벨을 합쳐줘서 학습